
# 🧪 **Results Analysis - Optimizations and Hypothesis Tests**  

The objective of this notebook is to analyze the intermediate results obtained by optimizations (with Optuna and Gridseacrch) of the three trained models: Generalized Linear Model (GLM), Generalized Additive Model (GAM) and Symbolic Regression (SR).

***

### 📚 **Importing libraries**

In [1]:
import optuna
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_timeline

import pandas as pd
import numpy as np
import plotly.express as px

import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from statsmodels.genmod.families import Gamma, Gaussian, InverseGaussian
from statsmodels.genmod.families.links import log as Log, identity as Identity, inverse_power as InversePower, inverse_squared as InverseSquared
from ucimlrepo import fetch_ucirepo
import os
import pickle


### **Generalized Linear Model (GLM) - Grid Search** 

In [2]:
df_gs_glm = pd.read_csv("../Other_files/gridsearch_glm_results.csv").drop(columns = "Unnamed: 0")
df_gs_glm.loc[df_gs_glm["RMSE"].idxmin()]

Family     Gaussian
Link            log
RMSE      15.102621
Name: 3, dtype: object

Grid Search, considering the limited number of families and link functions, shows as the best technique for optimizing GLM without clustering. 

### **Generalized Linear Model (GLM) with clustering - Bayesian Search** 

In [3]:
study_name = "optimization_clusters_glm_teste21_07_2025_16_46_28"
storage_url = "sqlite:///../Optuna_files/optimization_glm_bayesian.db"

study_glm_bayesian = optuna.load_study(study_name=study_name, storage=storage_url)


##### **Best trial**

In [4]:
print(f"The best trial had RMSE equal to {study_glm_bayesian.best_value:.2f}")

The best trial had RMSE equal to 13.73


In [5]:
print(f"The best params were: {study_glm_bayesian.best_params}")

The best params were: {'clusterer': 'birch', 'n_clusters_birch': 10, 'branching_factor': 700, 'threshold': 0.18251719168858943, 'distribution': 'gaussian', 'link_gaussian': 'identity'}


The link function was different from the optimization without clustering ("log" and "identity", respectively)

##### **Optimization history**

First, it´s necessary to remove outliers and concat two params that are used for the same thing: "params_n_clusters" and "params_n_clusters_bkmeans". Both are used to count the number of clusters, but for different clusteres.


In [6]:
df_glm = study_glm_bayesian.trials_dataframe()
df_filter_glm = df_glm[(df_glm["state"] != "PRUNED") & (df_glm["value"] <= 10e1)]

df_filter_glm["n_clusters"] = (
    df_filter_glm["params_n_clusters"]
    .fillna(df_filter_glm["params_n_clusters_bkmeans"])
    .fillna(df_filter_glm["params_n_clusters_birch"])
)

C:\Users\julia24002\AppData\Local\Temp\ipykernel_33188\3627378307.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filter_glm["n_clusters"] = (


This plot shows the study optimization history, by showing the objective value for each Optuna trial. Besides that, we can see the clusterer that was used and the number of clusters chosen.

In [7]:
fig = px.scatter(df_filter_glm, 
                 x="number", 
                 y="value", 
                 symbol="params_clusterer",
                 color="n_clusters",
                 title="Optimization History - GLM (Bayesian Search)",)
fig.update_traces(marker=dict(size=12))
fig.update_coloraxes(
    colorbar_title_side="top",
    colorbar=dict(len=0.5)  
)

fig.show()

It´s possible to notice that trials with Birch presented better results and the number of clusteres used varied between one and eleven.

##### **Hyperparameter Importances**

Plot hyperparameter importances based on the Optuna attempts.

In [8]:
plot_param_importances(study_glm_bayesian)

Clustered shows off as the most important feature, with 96% of contribution.

In [9]:
plot_timeline(study_glm_bayesian)


It is noticeable that, in the beginning, the algorithm faced difficulties in generating valid trials. However, as the process progressed, several complete trials were successfully obtained.

### **Generalized Linear Model (GLM) with clustering - Random Search** 

In [10]:
study_name = "optimization_clusters_glm_teste21_07_2025_16_50_13"
storage_url = "sqlite:///../Optuna_files/optimization_glm_random_search.db"

study_glm_rs = optuna.load_study(study_name=study_name, storage=storage_url)

##### **Best trial**

In [11]:
print(f"The best trial had RMSE equal to {study_glm_rs.best_value:.2f}")

The best trial had RMSE equal to 13.75


In [12]:
print(f"The best params were: {study_glm_rs.best_params}")

The best params were: {'clusterer': 'birch', 'n_clusters_birch': 11, 'branching_factor': 990, 'threshold': 0.3102963536234391, 'distribution': 'gaussian', 'link_gaussian': 'identity'}


##### **Optimization history**

In [13]:
df_glm = study_glm_bayesian.trials_dataframe()

df_filter_glm = df_glm[(df_glm["state"] != "PRUNED") & (df_glm["value"] <= 10e1)]

df_filter_glm["n_clusters"] = (
    df_filter_glm["params_n_clusters"]
    .fillna(df_filter_glm["params_n_clusters_bkmeans"])
    .fillna(df_filter_glm["params_n_clusters_birch"])
)


C:\Users\julia24002\AppData\Local\Temp\ipykernel_33188\1630359947.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [14]:
fig = px.scatter(df_filter_glm, 
                 x="number", 
                 y="value", 
                 symbol="params_clusterer",
                 color="n_clusters",
                 title="Optimization History - GLM (Random Search)",)
fig.update_traces(marker=dict(size=12))
fig.update_coloraxes(
    colorbar_title_side="top",
    colorbar=dict(len=0.5)  
)

fig.show()


Now we can clearly see that most attempts are still close to 15, which makes sense considering that the best attempt reached a value of 13.75. Besides that, it is evident that the prefered cluster was Birch and more clusters,, in general, led to a better result.

##### **Hyperparameter Importances**

Plot hyperparameter importances based on the Optuna attempts.

In [15]:
plot_param_importances(study_glm_rs)

Again, clusterer seems like the most important feature for Optuna Optimization.

##### **Timeline plot**

Plot the timeline of a study, indicating complete and pruned trials.

In [16]:
plot_timeline(study_glm_rs)


There were many pruned trials, likely due to a combination of random sampling and a numerically unstable model.

### **Generalized Linear Model (GAM) - Bayesian Search** 

In [17]:
study_name = "optimization_gam_teste25_09_2025_21_56_16" 

storage_url = f"sqlite:///../Optuna_files/optimization_gam_without_clustering.db"

study_gam = optuna.load_study(study_name=study_name, storage=storage_url)

#### **Best trial**

In [18]:
print(f"The best trial had RMSE equal to {study_gam.best_value:.2f}")

The best trial had RMSE equal to 13.14


In [19]:
print(f"The best params were {study_gam.best_params}")

The best params were {'distribution': 'normal', 'link_gaussian': 'identity', 'lam': 0.4122645931880113, 'n_splines': 25}


#### **Optimization history**

In [20]:
df_gam = study_gam.trials_dataframe()
df_filter_gam = df_gam[(df_gam["state"] != "PRUNED") & (df_gam["value"] <= 10e1)]

In [21]:
fig = px.scatter(df_filter_gam, 
                 x="number", 
                 y="value", 
                 symbol="params_distribution",
                 color="params_n_splines",
                 title="Optimization History - GAM (Bayesian Search)",)
fig.update_traces(marker=dict(size=12))
fig.update_coloraxes(
    colorbar_title_side="top",
    colorbar=dict(len=0.5)  # reduz altura da barra de cor
)


fig.show()

From the graph, it is possible to observe that a larger number of splines led to a lower RMSE. In addition, the normal distribution was preferred compared to the gamma distribution.

#### **Hyperparameter importance**

In [22]:
plot_param_importances(study_gam)

In general, the optimized parameter was lam - what makes sense considering that lam has a infinite number of possibilities (continuous float).

#### **Timeline plot**

In [23]:
plot_timeline(study_gam)

Many complete trials, indicating that the limitation of possible combinations between distribution and link functions was effective for avoid pruned trials.

### **Generalized Additive Model (GAM) with clustering - Bayesian Search**

In [24]:
study_name = "optimization_clusters_gam_teste21_07_2025_16_52_45" 

storage_url = f"sqlite:///../Optuna_files/optimization_gam_bayesian.db"

study_gam_bs = optuna.load_study(study_name=study_name, storage=storage_url)

#### **Best trial**

In [25]:
print(f"The best trial had RMSE equal to {study_gam_bs.best_value:.2f}")

The best trial had RMSE equal to 10.21


In [26]:
print(f"The best params were {study_gam_bs.best_params}")

The best params were {'clusterer': 'bisecting_kmeans', 'n_clusters_bkmeans': 24, 'bisecting_strategy': 'largest_cluster', 'distribution': 'normal', 'link_gaussian': 'identity', 'lam': 0.2457079020965402, 'n_splines': 20}


#### **Optimization history**

In [27]:
df_gam = study_gam_bs.trials_dataframe()
df_filter_gam = df_gam[(df_gam["state"] != "PRUNED") & (df_gam["value"] <= 10e1)]

df_filter_gam["n_clusters"] = (
    df_filter_gam["params_n_clusters"]
    .fillna(df_filter_gam["params_n_clusters_bkmeans"])
    .fillna(df_filter_gam["params_n_clusters_birch"])
)

C:\Users\julia24002\AppData\Local\Temp\ipykernel_33188\3420420207.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [28]:
fig = px.scatter(df_filter_gam, 
                 x="number", 
                 y="value", 
                 symbol="params_clusterer",
                 color="n_clusters",
                 title="Optimization History - GAM (Bayesian Search)",)
fig.update_traces(marker=dict(size=12))
fig.update_coloraxes(
    colorbar_title_side="top",
    colorbar=dict(len=0.5)  # reduz altura da barra de cor
)


fig.show()

The values are lower compared to those obtained with the GLM, which justifies the better results observed. Bisecting k-means emerges as the clustering method that yields the best performance. Additionally, there are fewer pruned trials. This can be explained by the number of parameters involved in the GAM: a suitable combination of distribution and link function — which are often responsible for model instability — may have been found early in the optimization process. As a result, subsequent trials were more focused on tuning numerical parameters, such as the regularization term (lambda) and the number of clusters, which have less impact on whether a trial is pruned.

#### **Hyperparameter Importance**

In [29]:
plot_param_importances(study_gam_bs)

In the case of GAM, the most important hyperparameter was lambda, which represents the regularization strenght.

#### **Timeline plot**

In [30]:
plot_timeline(study_gam_bs)

The time was longer compared with GLM, but there were fewer pruned trials.

### **Generalized Additive Model (GAM) with clustering - Random Search**

In [31]:
study_name = "optimization_clusters_gam_teste31_07_2025_09_51_57" 

storage_url = f"sqlite:///../Optuna_files/optimization_gam_random_search.db"

study_gam_rs = optuna.load_study(study_name=study_name, storage=storage_url)

##### **Best trial**

In [32]:
print(f"The best trial had RMSE equal to {study_gam_rs.best_value:.2f}")

The best trial had RMSE equal to 10.95


In [33]:
print(f"The best params were {study_gam_rs.best_params}")

The best params were {'clusterer': 'birch', 'n_clusters_birch': 25, 'branching_factor': 788, 'threshold': 0.5470948785370666, 'distribution': 'normal', 'link_gaussian': 'identity', 'lam': 0.7163145957304842, 'n_splines': 22}


##### **Optimization history**

In [34]:
df_gam = study_gam_rs.trials_dataframe()
df_filter_gam = df_gam[(df_gam["state"] != "PRUNED") & (df_gam["value"] <= 10e1)]

df_filter_gam["n_clusters"] = (
    df_filter_gam["params_n_clusters"]
    .fillna(df_filter_gam["params_n_clusters_bkmeans"])
    .fillna(df_filter_gam["params_n_clusters_birch"])
    )

C:\Users\julia24002\AppData\Local\Temp\ipykernel_33188\2306779367.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [35]:
fig = px.scatter(df_filter_gam, 
                 x="number", 
                 y="value", 
                 symbol="params_clusterer",
                 color="n_clusters",
                 title="Optimization History - GAM (Random Search)",)
fig.update_traces(marker=dict(size=12))
fig.update_coloraxes(
    colorbar_title_side="top",
    colorbar=dict(len=0.35)  # reduz altura da barra de cor
)


fig.show()

For the first time affinity propagation was chosen as clusterer and result in a valid trial. However, this type of clustering didn´t provided a good result at all.

##### **Hyperparameter Importances**

In [36]:
plot_param_importances(study_gam_rs)

As the result showed by bayesian search, lam was the most important hyperparameter.

##### **Timeline plot**

In [37]:
plot_timeline(study_gam_rs)

This type of search resulted in more pruned trials. Probably the random search resulted in more pruned trials. 

### **Symbolic Regression (SR) with clustering - Bayesian Search**

In [38]:
study_name = "optimization_clusters_gam_teste15_07_2025_15_40_36" 
# The first version of script_sr.py defined the name with "gam" word. This was fixed in the new version!
from pathlib import Path

storage_url = f"sqlite:///../Optuna_files/optimization_sr.db"


study_sr = optuna.load_study(study_name=study_name, storage=storage_url)

#### **Best trial**

In [39]:
print(f"The best trial had RMSE equal to {study_sr.best_value:.2f}")

The best trial had RMSE equal to 17.49


In [40]:
print(f"The best params were: {study_sr.best_params}")

The best params were: {'clusterer': 'bisecting_kmeans', 'n_clusters_bkmeans': 17, 'bisecting_strategy': 'largest_cluster', 'n_iterations': 23, 'maxsize': 29, 'maxdepth': 14, 'n_binary_operators': 9, 'binary_operators_0': '*', 'binary_operators_1': '^', 'binary_operators_2': '*', 'binary_operators_3': '*', 'binary_operators_4': '+', 'binary_operators_5': '*', 'binary_operators_6': '+', 'binary_operators_7': '+', 'binary_operators_8': '-', 'n_unary_operators': 19, 'unary_operators_0': 'sin', 'unary_operators_1': 'sin', 'unary_operators_2': 'neg', 'unary_operators_3': 'tan', 'unary_operators_4': 'cube', 'unary_operators_5': 'cos', 'unary_operators_6': 'sqrt', 'unary_operators_7': 'inv', 'unary_operators_8': 'sin', 'unary_operators_9': 'tan', 'unary_operators_10': 'abs', 'unary_operators_11': 'exp', 'unary_operators_12': 'exp', 'unary_operators_13': 'log', 'unary_operators_14': 'log', 'unary_operators_15': 'sqrt', 'unary_operators_16': 'abs', 'unary_operators_17': 'neg', 'unary_operators_1

#### **Optimization story**

In [41]:
df_sr = study_sr.trials_dataframe()
df_filter_sr = df_sr[(df_sr["state"] != "PRUNED") & (df_sr["value"] <= 10e1)]


df_filter_sr["n_clusters"] = df_filter_sr["params_n_clusters"].fillna(df_filter_sr["params_n_clusters_bkmeans"])

C:\Users\julia24002\AppData\Local\Temp\ipykernel_33188\917952740.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [42]:
fig = px.scatter(df_filter_sr, 
                 x="number", 
                 y="value", 
                 symbol="params_clusterer",
                 color="n_clusters",
                 title="Optimization History - SR",)
fig.update_traces(marker=dict(size=12))
fig.update_coloraxes(
    colorbar_title_side="top",
    colorbar=dict(len=0.6)  # reduz altura da barra de cor
)

fig.show()

Fewer attempts were pruned, compared to GLM and GAM. Addiotionally, more trials were performed with bisecting_kmeans, although kmeans still dominates the trials. 

##### **Hyperparameter Importances**

In [43]:
plot_param_importances(study_sr)

Maxsize appears as the most important feature, being followed by n_iterations and n_features. It´s difficult to analyse how unary operators 1 was the second most important feature, but it´s possible to notice that unary operators were most import for the predictions that binary ones.

#### **Timeline plot**

In [44]:
plot_timeline(study_sr)

[W 2025-10-05 08:34:37,324] The start and end times for Trial 100 seem to be reversed. The start time is 2025-07-16 09:50:27.571957 and the end time is 2025-07-16 03:39:39.144938.


Also many pruned trials were observed. A mistake led Optuna to consider a trial as "Running" for a long time, but we can disgard this trial for the analysis.

### **Hypothesis Test: Proving $H_2$**

In this project, our second objective is to verify whether clustering can lead models to better performance. Based on this, we can perform a t-Student test for the GLM and GAM results. The hypotheses are:

* $H_0$: The difference between the results of the test without clustering and with clustering is not statistically significant;

* $H_1$: The difference between the results of the test without clustering and with clustering is statistically significant.

First of all, we need to train the models with the optimizated hyperparameters.

##### **Importing data**

In [63]:
from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split
import numpy as np 

superconductivity_data = fetch_ucirepo(id=464)

X = superconductivity_data.data.features
y = superconductivity_data.data.targets
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.1, random_state=1702
    )

y_test = np.clip(y_test, 1e-6, None)
y_train = np.clip(y_train, 1e-6, None)


##### **GLM**

In [102]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
import statistics as st

kf = KFold(n_splits=10, shuffle=True, random_state=1203)
cv_glm = {}

for split, (train_idx, test_idx) in enumerate(kf.split(X_train)):
    X_train_cv, X_test_cv = X_train.iloc[train_idx], X_train.iloc[test_idx]
    y_train_cv, y_test_cv = y_train.iloc[train_idx], y_train.iloc[test_idx]

    model = sm.GLM(y_train_cv, X_train_cv, family=sm.families.Gaussian(link=sm.families.links.Log()))
    glm = model.fit()

    y_pred = glm.predict(X_test_cv)

    rmse = np.sqrt(mean_squared_error(y_test_cv, y_pred))
    cv_glm[split] = rmse

fold_values = list(cv_glm.values())
cv_glm["Mean"] = st.mean(fold_values)
cv_glm["Median"] = st.median(fold_values)

In [103]:
cv_glm

{0: 15.219201215912985,
 1: 15.887320764628733,
 2: 14.801496609323491,
 3: 15.111330863522138,
 4: 15.435506263230575,
 5: 15.061092332896571,
 6: 14.757015924735084,
 7: 15.321808183684489,
 8: 14.96579735927354,
 9: 15.667944468914655,
 'Mean': 15.222851398612226,
 'Median': 15.165266039717562}

In [104]:
glm_folds = list(cv_glm.values())[:-2]

##### **Clustering GLM**

In [84]:
import sys
sys.path.append("../Main_Scripts")

from script_glm import Clustering_GLM, cross_validation
from sklearn.cluster import Birch
from statsmodels.genmod.families.links import Identity


clusterer_glm_cluster = Birch(n_clusters=10,
                            branching_factor=700,
                            threshold=0.18251719168858943)

cv_glm_results = cross_validation(
    X = X_train, 
    y = y_train, 
    clusterer = clusterer_glm_cluster, 
    distribution=sm.families.Gaussian(link=Identity()),   
    random_seed=1203, 
    n_splits = 10         
)
                                                          


C:\Users\julia24002\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:2749: UserWarning:

X does not have valid feature names, but Birch was fitted with feature names

C:\Users\julia24002\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:2749: UserWarning:

X does not have valid feature names, but Birch was fitted with feature names

C:\Users\julia24002\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:2749: UserWarning:

X does not have valid feature names, but Birch was fitted with feature names

C:\Users\julia24002\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:2749: UserWarning:

X does not have valid feature names, but Birch was fitted with feature names

C:\Users\julia24002\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:2749: UserWarning:

X does not have valid feature names, but Birch was fitted with feature names

C:\Users\julia24002\AppData\Roaming

In [105]:
cv_glm_results

{0: 14.397403738217845,
 1: 14.326387951857429,
 2: 13.039220010058093,
 3: 13.4339012462349,
 4: 13.333216345787442,
 5: 13.532512296746381,
 6: 13.374314592668302,
 7: 14.378108224732848,
 8: 18.778959810379742,
 9: 13.9163889484636,
 'Mean': 14.251041316514659,
 'Median': 13.72445062260499}

In [106]:
glm_clustering_folds = list(cv_glm_results.values())[:-2]

##### **Hypothesis Test**

First of all, we need to define confidence and significance for doing the hypothesis tests. 

In [109]:
CONFIDENCE = 0.95

SIGNIFICANCE = 1 - CONFIDENCE

In [110]:
from scipy import stats

Now, we can check if data follow a normal distribution (If both data follow this distribution, we can use student test), using Shapiro Wilk test.

In [113]:
shapiro_result1 = stats.shapiro(glm_folds)

pvalor = shapiro_result1.pvalue

print(pvalor)

0.7849799533117592


In [115]:
shapiro_result2 = stats.shapiro(glm_clustering_folds)

pvalor = shapiro_result2.pvalue

print(pvalor)

0.0001517214612505275


Since the cross-validation results do not follow a normal distribution, it is possible to apply the Wilcoxon signed-rank test, a non-parametric alternative to the paired t-test (paired because data is the same). This test evaluates whether the median difference between the paired results of both models is statistically significant.

The hypotheses are defined as follows:

* $H_0$: The median of the differences between the cross-validation results of both models is equal to zero;

* $H_1$ : The median of the differences between the cross-validation results is greater than zero, so the distribution from which glm_folds was sampled is stochastically greater than that of glm_clustering_folds.

In [136]:
wilcoxon = stats.wilcoxon(glm_folds, glm_clustering_folds, alternative="greater")

pvalor = wilcoxon.pvalue

pvalor

0.0419921875

So we accepted $H_0$ (0.042 < 0.05): The median of the difference between the CV results of the model with and without clustering is equal to 0, which means that clustering didn´t impact on the results.

##### **GAM**

In [77]:
import pygam
from pygam import s
import statistics as st

cv_gam = {}

kf = KFold(n_splits=10, shuffle=True, random_state=1203)

split = 0
    
for train_idx, test_idx in kf.split(X_train):
        
    X_train_cv, X_test_cv = X_train.iloc[train_idx], X_train.iloc[test_idx]
    y_train_cv, y_test_cv = y_train.iloc[train_idx], y_train.iloc[test_idx]
            
    terms = s(0)
        
    for i in range(1, X_train.shape[1]):
        terms += s(i)
            
    model_gam = pygam.pygam.GAM(terms, 
                                distribution="normal", 
                                link="identity", 
                                lam=0.2457079020965402, 
                                n_splines=20)
    
    model_gam.fit(X_train_cv, y_train_cv)

    # Make the prediction
    y_pred = model_gam.predict(X_test_cv)

    # Calculate RMSE
    rmse = root_mean_squared_error(y_test_cv, y_pred)

    cv_gam[split] = rmse
    split += 1

fold_values = list(cv_gam.values())
cv_gam["Mean"] = st.mean(fold_values)
cv_gam["Median"] = st.median(fold_values)


In [95]:
cv_gam

{0: 13.539436279640547,
 1: 14.205275601938848,
 2: 13.080293677940684,
 3: 13.872400786204935,
 4: 13.373020621059746,
 5: 13.330939292961489,
 6: 13.518339627490553,
 7: 14.38589119183338,
 8: 13.329350434516439,
 9: 14.024487512167779,
 'Mean': 13.66594350257544,
 'Median': 13.528887953565551}

In [123]:
gam_folds = list(cv_gam.values())[:-2]

##### **Clustering GAM**

In [66]:
import sys
sys.path.append("../Main_Scripts")

from script_gam import Clustering_GAM, cross_validation
from sklearn.cluster import BisectingKMeans

clusterer_gam_cluster = BisectingKMeans(n_clusters=24,
                                        init = "k-means++",
                                        random_state = 1203,
                                        bisecting_strategy="largest_cluster")


clustering_gam = Clustering_GAM(clusterer_gam_cluster, "normal", "identity", 20, 0.2457079020965402)
cv_gam_results = cross_validation(X_train, y_train, clusterer_gam_cluster, "normal", "identity", 0.2457079020965402, 20, n_splits=10)

C:\Users\julia24002\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:2749: UserWarning:

X does not have valid feature names, but BisectingKMeans was fitted with feature names

C:\Users\julia24002\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:2749: UserWarning:

X does not have valid feature names, but BisectingKMeans was fitted with feature names

C:\Users\julia24002\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:2749: UserWarning:

X does not have valid feature names, but BisectingKMeans was fitted with feature names

C:\Users\julia24002\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:2749: UserWarning:

X does not have valid feature names, but BisectingKMeans was fitted with feature names

C:\Users\julia24002\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:2749: UserWarning:

X does not have valid feature names, but BisectingKMeans was fitted with 

In [97]:
cv_gam_results

{0: 10.72946572225453,
 1: 12.12116572897272,
 2: 11.622169616410394,
 3: 11.054541056525295,
 4: 11.842880016253106,
 5: 11.905701532730584,
 6: 12.207463738928817,
 7: 11.689702113423673,
 8: 10.338568725817812,
 9: 11.693860052021977,
 'Mean': 11.52055183033389,
 'Median': 11.691781082722825}

In [124]:
gam_clustering_folds = list(cv_gam_results.values())[:-2]

##### **Hypothesis test**

We can made the same procedure from GLM, following the hypothesis:

* $H_0$: The median of the differences between the cross-validation results of both models is equal to zero;

* $H_1$ : The median of the differences between the cross-validation results is greater than zero, so the distribution from which gam_folds was sampled is stochastically greater than that of gam_clustering_folds.

In [135]:
wilcoxon2 = stats.wilcoxon(gam_folds, gam_clustering_folds, alternative="greater")

pvalor = wilcoxon2.pvalue

pvalor

0.0009765625

Again, we can accept $H_0$ (0.001 < 0.05), so the median of the difference between the CV results of the model with and without clustering is equal to 0. In other words, clustering didnt´t impact in the final results. 

### 🤔 **Conclusions**

At the end of the analysis, we can conclude that all three tests resulted in studies with a high number of pruned trials. This likely occurred due to two main factors:
*  First, the supervised models used (GLM, GAM, and Symbolic Regression) can be numerically unstable, especially GLM and GAM. These models are prone to divergence depending on the chosen distribution and link function; 
* Second, the frequent selection of Birch and Bisecting KMeans as preferred clustering methods suggests that other clustering algorithms may have led to unstable trials as well.


Regarding feature importance, in the GLM trials, the most influential parameter was the distribution, which aligns with the observed numerical instability. For GAM and Symbolic Regression, the most important parameters were, respectively, the number of splines and maxsize, both of which directly influence model complexity. In general, as observed in the notebook "Symbolic Regression Equations", more complex models tended to deliver better performance.

Besides that, by the hypothesis tests made, we could conclude that clustering didn´t impact in models performances.


### 🗃️ **References!**

[1] Quick Visualization for Hyperparameter Optimization Analysis — Optuna 4.4.0 documentation. Available: <https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html>. Acess in: 9 jul. 2025.

